Creacion tabla *Caracteristicas_fisicas_bloqueos2*

A partir de el Analisis Exploratorio iniciamos las tareas de consolidar vistas que nos permitan organizar los datos en tablas que nos sirvan como insumo para generar nuestra tabla de analitica final. durante esta tarea generamos las consultas que presentamos a continuación.

Aquí generamos la tabla Caracteristicas_fisicas_bloqueos2 el objetivo garantizar que los campo *team_id* no contenga datos nulos dado que será usado como clave foranea para generar asocianos con nustras otras tablas, aquí se agrupan los datos temporada, jugador, el identificador del equipo, id del jugador y caracteristicas fisicas de los jugadores. se realiza el conteo del total de bloqueos y se asocia con las caracteristicas fisicas de los jugadores(estatura, peso, alcance, etc), ordenados por la temporada, el equipo, y ordena la salida de mayor a menor con respecto al total de bloqueos. Cabe anotar que estamos extrayendo datos de la tabla *play_by_play* y concatenandolos con datos de la vista *total_bloqueos_por_jugador_y_temporada_2*

In [ ]:
create table Caracteristicas_fisicas_bloqueos2 as
SELECT
    tbpjyt2.season,
    tbpjyt2.jugador,
    tbpjyt2.team_id,
    tbpjyt2.team_name,
    tbpjyt2.id_jugador,
    COUNT(*) AS total_blocks,
    dcs.height_wo_shoes,
    dcs.weight,
    dcs.wingspan,
    dcs.standing_reach,
    dcs.max_vertical_leap
FROM
    total_bloqueos_por_jugador_y_temporada_2 tbpjyt2
JOIN
    common_player_info_202405271940 cpi ON tbpjyt2.id_jugador  = cpi.person_id
JOIN
    draft_combine_stats_202405271940 dcs ON tbpjyt2.id_jugador  = dcs.player_id
GROUP BY
    tbpjyt2.season, tbpjyt2.jugador, tbpjyt2.team_name, tbpjyt2.id_jugador, dcs.height_wo_shoes, dcs.weight, dcs.wingspan, dcs.standing_reach, dcs.max_vertical_leap,tbpjyt2.team_id
ORDER BY
    tbpjyt2.season, tbpjyt2.team_name, total_blocks DESC;

 Creacion tabla *total_bloqueos_por_jugador_y_temporada_2*

 En esta consulta estamos generando una vista que nos muestra de manera agrupada el jugador, el nombre del equipo, el id del jugador el id del equipo para conoce el numero de bloqueos de cada jugador por temporada. ordenamos los resultados por temporada, el nombre del equipo y se presentan los datos ordenados de mayor a menor valor con respecto al numero de bloqueos. en este caso estamos gnerando una consulta sobre nuestra vista *historial_bloqueos* cuya generacion se muetra subsecuente al codigo que presentamos a continuación.

In [ ]:
CREATE TABLE total_bloqueos_por_jugador_y_temporada_2 AS
SELECT
    season,
    jugador,
    id_jugador,
    team_name,hb.team_id,
    COUNT(*) AS total_blocks
FROM
    historial_bloqueos hb
GROUP BY
    season, jugador, id_jugador, team_name,team_id
ORDER BY
    season, team_name, total_blocks DESC;


Usando la *tabla play_by_play* como insumo principal, se genera la vista *historial_bloqueos* obteniendo los datos id de juego(*game_id*), temporada(*season*), jugador, id del jugador y la posicion en la que se desmpeña el jugador(*position*), se realiza el conteo de la palabara *BLOCK* dentro del campo *description* de la tabla mencionada y se evalua para las tempoaradas a partir de 2020, como una manera de generar una reduccion de datos y realizar el proceso de analitica sobre 3 temporadas. con el objetivo de brindar una vision clara de los datos que se extraen de otras tablas, que nos permiten configurar la vista, las otras tablas usadas han sido *game_sumary*, *common_player_info*, *draft_combine_stats*, se mantiene la idea de usar los campos *season*, *player_id* y *team_id* como claves que nos permitan generar relacion con otras tablas.

In [ ]:
CREATE TABLE Bloqueos AS
SELECT
    pbp.game_id,
    gs.season,
    pbp.homedescription AS description,
    pbp.player1_name AS Jugador,
    pbp.player1_id AS ID_jugador,
    SPLIT_PART(pbp.player1_team_id::TEXT, '.', 1) AS team_id,
    team_name,
    cpi.position AS posicion
FROM
    play_by_play_202405271940 pbp
JOIN
    game_summary_202405271940 gs ON pbp.game_id = gs.game_id
JOIN
    common_player_info_202405271940 cpi ON pbp.player1_id = cpi.person_id
JOIN
    draft_combine_stats_202405271940 dcs ON pbp.player1_id = dcs.player_id
WHERE
    pbp.homedescription LIKE '%BLOCK%'
    AND gs.season::INTEGER >= 2020;
UNION ALL

SELECT
    pbp.game_id,
    gs.season,
    pbp.visitordescription AS description,
    pbp.player3_name AS Jugador,
    pbp.player3_id AS ID_jugador,
    SPLIT_PART(pbp.player3_team_id::TEXT, '.', 1) AS team_id,
    team_name,
    cpi.position AS posicion
FROM
    play_by_play_202405271940 pbp
JOIN
    game_summary_202405271940 gs ON pbp.game_id = gs.game_id
JOIN
    common_player_info_202405271940 cpi ON pbp.player3_id = cpi.person_id
JOIN
    draft_combine_stats_202405271940 dcs ON pbp.player3_id = dcs.player_id
WHERE
    pbp.visitordescription LIKE '%BLOCK%'
    AND gs.season::INTEGER >= 2020
ORDER BY
    pbp.wctimestring,
    pbp.pctimestring;